In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
print(data)

In [3]:
data.info()

In [4]:
data.describe()

In [5]:
data = pd.get_dummies(data)
data.head()

In [6]:
data['charges'].plot(kind = 'hist')

In [7]:
data["log_charges"] = np.log2(data["charges"] + 0.1)
data["log_charges"].plot(kind = 'hist')
data = data.drop(["charges"], axis=1)

In [8]:
y = data['log_charges']
X = data.drop(['log_charges'], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state = 1)

In [10]:
plt.scatter(data["age"], data["bmi"]);

In [11]:
# Calculate and plot
numerical = list(
    set(data.columns)
    - {
        "sex_male",
        "smoker_no",
        "region_southwest",
        "region_northwest",
        "region_southeast",
        "region_northeast"
    }
)
corr_matrix = data[numerical].corr()
sns.heatmap(corr_matrix, annot=True);

**From here we can conclude that smoking has direct correlation(0.67) with the medical charges, and then age has a correlation of 0.53**

In [12]:
%config InlineBackend.figure_format = 'png'
sns.pairplot(data[numerical]);

In [13]:
sns.scatterplot(data=data, x="age",y="log_charges", hue="sex_male")

# Standard scaling

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn import metrics
scores = []

# **Linear Regression:**

In [16]:
modelLinR= LinearRegression(normalize = True)
modelLinR.fit(X_train_scaled,y_train)
y_pred_lin = modelLinR.predict(X_test_scaled)
error_LinR = metrics.mean_absolute_error(y_pred_lin, y_test)
print(error_LinR)
scores.append({"Linear Regression", error_LinR})

# **Ridge Regression:**

In [17]:
modelRidR= Ridge()
modelRidR.fit(X_train_scaled,y_train)
y_pred_rid = modelRidR.predict(X_test_scaled)
error_RidR = metrics.mean_absolute_error(y_pred_rid, y_test)
print(error_RidR)
scores.append({"Ridge Regression", error_RidR})

# **Random Forest Regression:**

In [18]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
model_rf = RandomForestRegressor(max_depth=15, n_estimators=10, max_features=5)
model_rf.fit(X_train_scaled, y_train)
y_pred_rf = model_rf.predict(X_test_scaled)
error_rf = metrics.mean_absolute_error(y_pred_rf, y_test)
print(error_rf)
scores.append({"Random Forest Regression", error_rf})

# **Gradient Boosting Regression:**

In [19]:
model_gb = GradientBoostingRegressor()
model_gb.fit(X_train_scaled, y_train)
y_pred_gb = model_gb.predict(X_test_scaled)
error_gb = metrics.mean_absolute_error(y_pred_gb, y_test)
print(error_gb)
scores.append({"Gradient Boosting Regression", error_gb})

# **Extra Trees Regression:**

In [20]:
model_et = ExtraTreesRegressor()
model_et.fit(X_train_scaled, y_train)
y_pred_et = model_et.predict(X_test_scaled)
error_et = metrics.mean_absolute_error(y_pred_et, y_test)
print(error_et)
scores.append({"Extra Trees Regression", error_et})

***From the above results, Gradient Boosting Regression works best for our dataset and has MAE equal to 0.245***